In [1]:
import pandas as pd
import geopandas as gpd

In [8]:
def merge_data_sources(acs5):
    '''
    Links acs5 data with transit score and job data. Calculates population 
    density and job density.
    Inputs:
        acs5 (pandas DataFrame)
    Outputs:
        (full pandas DataFrame with transit score data)
    '''
    #Extracting census tract ID
    acs5['tract_GEO_ID'] = acs5['GEO_ID'].apply(lambda x: x[9:])

    #Loading tracts
    tracts = gpd.read_file('shape_tracts/tl_2018_17_tract.shp')
    tracts = tracts[['GEOID', 'NAMELSAD', 'ALAND', 'geometry']] \
                        .rename(columns={'GEOID': 'tract_GEO_ID', 'NAMELSAD': 'tract_name',
                       'ALAND': 'tract_area'})

    #Loading places
    places = gpd.read_file('shape_places/tl_2018_17_place.shp')
    places = places[['GEOID', 'NAME', 'NAMELSAD', 'geometry']] \
                        .rename(columns={'GEOID': 'place_GEO_ID', 'NAME': 'place_name',
                       'NAMELSAD': 'place_name_and_type'})

    #Merging tracts and places
    tracts_places = gpd.sjoin(tracts, places, how="inner", op="intersects")

    #Merging acs data with traces/places
    df = pd.merge(acs5, tracts_places, left_on='tract_GEO_ID', right_on='tract_GEO_ID')

    #Importing transit score csv and merging
    ts = pd.read_csv('transit_score.csv').rename(columns={'nearby_routes': 'num_nearby_routes', \
         'bus': 'num_bus_routes', 'rail': 'num_rail_routes', 'other': 'num_other_routes', \
         'city': 'city_from_ts', 'description': 'transit_description', 'summary': 'transit_summary', \
         'Lat': 'lat', 'Lon': 'lon'})
    ts['tsplace_GEO_ID'] = ts['GEO_ID'].apply(lambda x: x[9:])
    ts = ts.drop(columns=['censusgeo', 'Place_Type', 'state', 'GEO_ID'])
    df = pd.merge(df, ts, how='inner', left_on='place_GEO_ID', right_on='tsplace_GEO_ID')

    #Importing jobs by tract and merging
    jobs = pd.read_csv('il_jobs_by_tract_2017.csv')
    jobs = jobs[['id', 'label', 'c000']] \
            .rename(columns={'id': 'job_tract_GEO_ID', 'label': 'job_tract_label',
                             'c000': 'num_jobs'})
    jobs['job_tract_GEO_ID'] = jobs['job_tract_GEO_ID'].astype(str)
    df = pd.merge(jobs, df, how='inner', left_on='job_tract_GEO_ID', right_on='tract_GEO_ID')

    #Averaging transit score for census tracts
    df = df.groupby('GEO_ID').mean().reset_index()

    #Calculating population density and job density
    df['job_density'] = df['num_jobs'] / ((df['tract_area'])/1000000)
    df['pop_density'] = df['race_total'] / ((df['tract_area'])/1000000)

    #Taking care of null values using pipline methods
    cols_with_null = explore_df_summary_stats(df)
    final_df, replacement = impute(df, cols_with_null)

    return final_df

In [2]:
acs5 = pd.read_pickle('raw_acs5.pkl')
from download import merge_data_sources
acs5.shape

(3123, 106)

In [3]:
final_df = merge_data_sources(acs5)

initial acs5 shape: 3123
inital number of colums: 107
--------------------------------------------------------------
df shape after merging with tracts_place: 6566
df number of colums: 114
--------------------------------------------------------------
df shape after mergin with transit score: 2097
df number of colums: 126
--------------------------------------------------------------
df shape after mergin with jobs data: 2094
df number of colums: 129
--------------------------------------------------------------
df shape after grouping: 1716
df number of colums: 115
df cols omitted becauseo of grouping: {'index', 'tsplace_GEO_ID', 'tract_name', 'job_tract_label', 'ws_link', 'place_name', 'place_name_and_type', 'transit_summary', 'place_GEO_ID', 'job_tract_GEO_ID', 'transit_description', 'tract_GEO_ID', 'city_from_ts', 'geometry'}
df num columns with zero population: 0


In [4]:
final_df.to_pickle('data.pkl')

In [2]:
data = pd.read_pickle('data.pkl')

In [7]:
data[data.isnull().any(axis=1)].values

array([['1400000US17097863005', 1.0, 0.0, 0.0, 0.0, 440.0, 138.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 242.0, 0.0, 0.0, 0.0, 380.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 0.0, 0.0, 47.0, 18.0,
        83.0, 9.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 440.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        376.0, 440.0, 64.0, 0.0, 0.0, 0.0, 0.0, 440.0, 0.0, -666666666.0,
        17.0, 0.0, 0.0, 110.0, 440.0, 0.0, 0.0, 0.0, 0.0, 46.0, 440.0,
        338.0, 17.0, 2018.0, 0.0, nan, nan, nan, 0.038636363636363635,
        0.0, 0.0, 0.0963855421686747, 0.0, 0.7831325301204819,
        0.12048192771084337, 0.0, 0.0, 0.7681818181818182,
        0.10454545454545454, 0.0, 0.14545454545454545, 1.0, 0.0, 0.0,
        0.0, 0.25, 0.0, 0.0, -666666666.0, nan, 1019796.0, 1164.0, 35.0,
        4.0, 3.0, 1.0, 0.0, 42.32525513, -87.84030557,
        0.9805882745176487, 431.45884078776544]], dtype=object)

In [11]:
data = data.fillna(0)
data[data.isnull().any(axis=1)]

,GEO_ID,num_jobs,car_avail_none,car_avail_total,commut_took_public_trans,commut_total,commute_time_10_to_14_min,commute_time_15_to_19_min,commute_time_20_to_24_min,commute_time_25_to_29_min,...,index_right,transit_score,num_nearby_routes,num_bus_routes,num_rail_routes,num_other_routes,lat,lon,job_density,pop_density


In [13]:
data[data['GEO_ID']=='1400000US17097863005'].values

array([['1400000US17097863005', 1.0, 0.0, 0.0, 0.0, 440.0, 138.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 242.0, 0.0, 0.0, 0.0, 380.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 0.0, 0.0, 47.0, 18.0,
        83.0, 9.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 440.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        376.0, 440.0, 64.0, 0.0, 0.0, 0.0, 0.0, 440.0, 0.0, -666666666.0,
        17.0, 0.0, 0.0, 110.0, 440.0, 0.0, 0.0, 0.0, 0.0, 46.0, 440.0,
        338.0, 17.0, 2018.0, 0.0, 0.0, 0.0, 0.0, 0.038636363636363635,
        0.0, 0.0, 0.0963855421686747, 0.0, 0.7831325301204819,
        0.12048192771084337, 0.0, 0.0, 0.7681818181818182,
        0.10454545454545454, 0.0, 0.14545454545454545, 1.0, 0.0, 0.0,
        0.0, 0.25, 0.0, 0.0, -666666666.0, 0.0, 1019796.0, 1164.0, 35.0,
        4.0, 3.0, 1.0, 0.0, 42.32525513, -87.84030557,
        0.9805882745176487, 431.45884078776544]], dtype=object)